In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats


In [14]:
df = pd.read_csv('../../data/sweep_data.csv')
df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp"], inplace=True)

In [15]:
df.head()

,amd,rcm,padding,matrix_name,partial_gauss,rank,tile_size,max_mode_size,n,z,r2mode6
0,False,False,3,ex15,3,19818.0,3.0,109.0,6867.0,98640.0,6.586863e+20
1,False,False,3,ex15,3,8771.0,7.0,109.0,6867.0,98640.0,1.290201e+20
2,False,False,3,ex15,3,6821.0,9.0,109.0,6867.0,98640.0,7.802883e+19
3,False,False,3,ex15,3,2921.0,21.0,109.0,6867.0,98640.0,1.430942e+19
4,False,False,3,ex15,3,539.0,63.0,109.0,6867.0,98640.0,4.872328e+17


In [16]:
baseline_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]

In [17]:
baseline_df.head(100)

,amd,rcm,padding,matrix_name,partial_gauss,rank,tile_size,max_mode_size,n,z,r2mode6
274,False,False,0,ex15,0,19825.0,3.0,109.0,6867.0,98671.0,6.591517e+20
275,False,False,0,ex15,0,8125.0,7.0,109.0,6867.0,98671.0,1.107148e+20
276,False,False,0,ex15,0,6175.0,9.0,109.0,6867.0,98671.0,6.394888e+19
277,False,False,0,ex15,0,2275.0,21.0,109.0,6867.0,98671.0,8.680041e+18
278,False,False,0,ex15,0,325.0,63.0,109.0,6867.0,98671.0,1.771437e+17
279,False,False,0,ex15,0,187.0,109.0,109.0,6867.0,98671.0,5.864651e+16
280,False,False,0,ex15,0,61.0,327.0,327.0,6867.0,98671.0,1.866307e+09
281,False,False,0,ex15,0,25.0,763.0,763.0,6867.0,98671.0,-1.513429e+09
282,False,False,0,ex15,0,19.0,981.0,981.0,6867.0,98671.0,1.643374e+09
283,False,False,0,ex15,0,7.0,2289.0,2289.0,6867.0,98671.0,-1.333853e+09


In [18]:
baseline_df_ex13 = baseline_df[baseline_df['matrix_name'] == "ex13"]
baseline_df_ex13.head(40)
# we got an integer overflow...

,amd,rcm,padding,matrix_name,partial_gauss,rank,tile_size,max_mode_size,n,z,r2mode6
498,False,False,0,ex13,0,22354.0,2.0,107.0,2568.0,75628.0,7.499169e+20
499,False,False,0,ex13,0,14978.0,3.0,107.0,2568.0,75628.0,3.366746e+20
500,False,False,0,ex13,0,9234.0,4.0,107.0,2568.0,75628.0,1.279624e+20
501,False,False,0,ex13,0,5456.0,6.0,107.0,2568.0,75628.0,4.467365e+19
502,False,False,0,ex13,0,3637.0,8.0,107.0,2568.0,75628.0,1.985131e+19
503,False,False,0,ex13,0,1990.0,12.0,107.0,2568.0,75628.0,5.943042e+18
504,False,False,0,ex13,0,843.0,24.0,107.0,2568.0,75628.0,1.066493e+18
505,False,False,0,ex13,0,82.0,107.0,107.0,2568.0,75628.0,1.009091e+16
506,False,False,0,ex13,0,34.0,214.0,214.0,2568.0,75628.0,1.558645e+09
507,False,False,0,ex13,0,22.0,321.0,321.0,2568.0,75628.0,-1.566708e+09


Show how the choice of tile size effects r2mode6 - show the 5 matrices on one plot together by different lines/colors.

use scatter since we have discrete values
x-axis: tile_size
y-axis (log?): r2mode6
legend: 5 matrix names

In [20]:
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species", symbol="species")
fig.update_layout(
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black') # Font color
)
fig.show()

fig.write_image("plots/fig1.pdf")